#Data Extraction for SEMRYE project

Extract word pairs of rhymes in poems. Current for the collections of Shakespeare and Housman available in https://github.com/sravanareddy/rhymedata, courtesy of Sravana Reddy.


## Output Format

Output format would be string tuples: 
**(word1 <str>, word2 <str>, rhyme pattern <str>, rhyme letter <str>, poetm title <str>, section id <str>,  author <str>)**

For example,

**Input:**

>TITLE A Lover's Complaint

>RHYME a b a b b c c

>From off a hill whose concave womb reworded

>A plaintful story from a sistering vale,

>My spirits to attend this double voice reworded,

>And down I laid to list the sad-tuned tale;

>Ere long espied a fickle maid full pale, 

>Tearing of papers, breaking rings a-twain,

>Storming her world with sorrow's wind and rain.

> ...

For the full scheme of the data, see: https://github.com/sravanareddy/rhymedata

** Output:**

*(reworded, reworded, ababbcc, a, A Lover's Complaint, 1, Shakespeare )*

*(vale, tale, ababbcc, b, A Lover's Complaint, 1,  Shakespeare )*

*(vale, pale, ababbcc, b, A Lover's Complaint, 1, Shakespeare )*

*(tale, pale, ababbcc, b, A Lover's Complaint, 1, Shakespeare )*


...

**Note1** For now, if a rhyme involves multiple words, like the **b** rhyem above, which as 3 words *vale*, *tale*, and *pale*. We extract all the pairs and keep the partial order, i.e.  *(vale, tale)*, *(tale, pale)* and *(vale, pale)* for the example

**Note2** a special scheme used in the original data is 
> Sometimes, we use a shorthand for the rhyme scheme, like 

>RHYME a a *

>This denotes the rhyme scheme aabbccdd...

We elaborate such patterns in this case.

** Note3** We may also want to keep a mapping from triple (poetm title <str>, section id <str>,  author <str>) to (section text, rhyme pattern)


## Code Design

### Format of input data
The format of original data is quite regular
Each file dedicates to one author, which is specified in the first line

**poem structure**:
>an empty line

>title

>sections*

**section strucre**:

>empty line

>rhyme pattern

>empty line

>poem line*

Poem lines are consecutive lines with content.

**Rhyme is at the last word of a poem line**


### Format of internal data

**collections**: a list of **poem**, which is a tuple of (**poem title**, **list_of_section**)
**list_of_section**: a list of **section**, each of which is a tuple (**section id**, **section text**, line_count)

#### Two outputs 

1. **word_pair_list**: a list of word pairs, each of the format **(word1 <str>, word2 <str>, rhyme pattern <str>, rhyme letter <str>, poetm title <str>, section id <str>**  note, no author information is here, but can be easily inserted

2. **section_map**: a dictionary. key: (poem title, section id)  value: tuple(section text, rhyme pattern)


### Functions
Looks like there is no need for internal states offered by class. So the code will be procedure-based.

def **poem extract (file_name)**:
    1. get the *author_name* string
    
    loops for finding the poem
        get the *poem title*
        main *section_count*
        
        loop for finding sections
            get the *rhyme string*
            get *section text* and line count
            add entry to *section_map*
            
            call **parse_rhyme_pattern**, get rhyme pattern
            call **pair_extraction**, get word pair tuple, insert to *word_pair_list*
            

return  word_pair_list, section_map, author_name 


def **parse_rhyme(rhyme string, line_count)**:
    return  new_rhyme_str, index_pair_list

Format of index_pair_list: (index1, index2, rhyme_letter)


def **pair_extraction(index_pair_list)** :
    return raw_pair_list
    
Format of each pair in raw_pair_list:  (word1, word2, letter)

